In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from diffusers import UNet2DModel, DDPMScheduler
from PIL import Image
import pandas as pd
import os

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [12]:
df = pd.read_parquet("../data/processed_sticker_dataset.parquet")
